In [1]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 3060 Laptop GPU
1


In [2]:
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.utils
from PIL import Image
import glob
import sys, os
sys.path.append(os.pardir) # 부모 디렉토리의 파일을 가져올 수 있도록 설정
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from collections import OrderedDict
import pickle
import torch
%matplotlib inline
import cv2
from imutils.video import VideoStream
from pathlib import Path
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import torch.nn as nn
from PIL import Image
from torch.autograd import Variable
import PIL.ImageOps
from torch import optim
import torch.nn.functional as F
import random
import numpy as np

In [3]:
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: c:\Users\yuddo\OneDrive\바탕 화면\temp\Personal_Project\3-1


In [5]:
user_dir = './data/user'

In [6]:
output_dir = './data/output'

In [7]:
def output_keypoints(frame, net, threshold, BODY_PARTS, now_frame, total_frame):
    global points
    
    # 입력 이미지 사이즈 정의
    image_height = 368
    image_width = 368
    
    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0/255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)
    
    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    print(f"============================== frame: {now_frame:.0f} / {total_frame:.0f} ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    return frame

In [8]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    return frame

In [9]:
def output_keypoints_with_lines_video(proto_file, weights_file, video_path, threshold, BODY_PARTS, POSE_PAIRS):

    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)
    
    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
    
    # GPU 사용
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    if capture.isOpened():
        fps = capture.get(cv2.CAP_PROP_FPS)
        frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        output = cv2.VideoWriter("result.avi", fourcc, fps, (frame_width, frame_height))
    
    while True:
        now_frame_boy = capture.get(cv2.CAP_PROP_POS_FRAMES)
        total_frame_boy = capture.get(cv2.CAP_PROP_FRAME_COUNT)

        if now_frame_boy == total_frame_boy:
            break

        ret, frame_boy = capture.read()
        frame_boy = output_keypoints(frame=frame_boy, net=net, threshold=threshold, BODY_PARTS=BODY_PARTS, now_frame=now_frame_boy, total_frame=total_frame_boy)
        frame_boy = output_keypoints_with_lines(frame=frame_boy, POSE_PAIRS=POSE_PAIRS)
        cv2.imshow("Output_Keypoints", frame_boy)
        output.write(frame_boy)
        
        if cv2.waitKey(10) == 27:  # esc 입력시 종료
            break

    capture.release()
    output.release()
    print("Output Video worked successfully")
    cv2.destroyAllWindows()


In [10]:
BODY_PARTS_MPI = {0: "Head", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                  5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                  10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "Chest",
                  15: "Background"}

POSE_PAIRS_MPI = [[0, 1], [1, 2], [1, 5], [1, 14], [2, 3], [3, 4], [5, 6],
                  [6, 7], [8, 9], [9, 10], [11, 12], [12, 13], [14, 8], [14, 11]]

BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]

BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

# 신경 네트워크의 구조를 지정하는 prototxt 파일 (다양한 계층이 배열되는 방법 등)
protoFile_mpi = "C:/Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_deploy_linevec.prototxt"
protoFile_mpi_faster = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_deploy_linevec_faster_4_stages.prototxt"
protoFile_coco = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\coco\pose_deploy_linevec.prototxt"
protoFile_body_25 = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose-master\models\pose\body_25\pose_deploy.prototxt"

# 훈련된 모델의 weight 를 저장하는 caffemodel 파일
weightsFile_mpi = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_iter_160000.caffemodel"
weightsFile_coco = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\coco\pose_iter_440000.caffemodel"
weightsFile_body_25 = "C:/Users/yuddo/Yuchul/Project/Bowling/openpose-master/models/pose/body_25/pose_iter_584000.caffemodel"

# 비디오 경로
man = "C:/Users/yuddo/Yuchul/Project/Bowling/data/pro/pro_1.mp4"

# 키포인트를 저장할 빈 리스트
points = []

## 아래 필요한 모델 주석 해제 후 사용!

#output_keypoints_with_lines_video(proto_file=protoFile_mpi_faster, weights_file=weightsFile_mpi, video_path=man,
#                                 threshold=0.1, BODY_PARTS=BODY_PARTS_MPI, POSE_PAIRS=POSE_PAIRS_MPI)

#output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=man,
                                  #threshold=0.1, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

#output_keypoints_with_lines_video(proto_file=protoFile_body_25, weights_file=weightsFile_body_25, video_path=man,
#                                 threshold=0.1, BODY_PARTS=BODY_PARTS_BODY_25, POSE_PAIRS=POSE_PAIRS_BODY_25)

In [19]:
user = "C:/Users/yuddo/Yuchul/Project/Bowling/data/user/3.mp4"

output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=user,
                                  threshold=0.15, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

============================== frame: 0 / 300 ==============================
[not pointed] Nose (0) => prob: 0.00993 / x: 516 / y: 208
[pointed] Neck (1) => prob: 0.50229 / x: 328 / y: 500
[pointed] RShoulder (2) => prob: 0.56581 / x: 563 / y: 542
[pointed] RElbow (3) => prob: 0.49469 / x: 516 / y: 1001
[pointed] RWrist (4) => prob: 0.21958 / x: 540 / y: 1043
[pointed] LShoulder (5) => prob: 0.53799 / x: 93 / y: 417
[not pointed] LElbow (6) => prob: 0.10572 / x: 70 / y: 876
[not pointed] LWrist (7) => prob: 0.04125 / x: 516 / y: 1001
[pointed] RHip (8) => prob: 0.25369 / x: 399 / y: 1210
[pointed] RKnee (9) => prob: 0.21204 / x: 352 / y: 1669
[not pointed] RAnkle (10) => prob: 0.01937 / x: 281 / y: 1878
[pointed] LHip (11) => prob: 0.23403 / x: 140 / y: 1168
[pointed] LKnee (12) => prob: 0.39100 / x: 211 / y: 1586
[not pointed] LAnkle (13) => prob: 0.06610 / x: 258 / y: 1878
[not pointed] REye (14) => prob: 0.04016 / x: 516 / y: 208
[not pointed] LEye (15) => prob: 0.01377 / x: 281 / y

In [16]:
def split_video_into_frames(video_path, output_folder):
    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)

    frame_count = 0

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        frame_path = f"{output_folder}/frame_{frame_count}.jpg"
        cv2.imwrite(frame_path, frame)

        frame_count += 1

    capture.release()

In [ ]:
split_video_into_frames()